# How to...validate banking IDs

This notebook shows how to use the entitymatching api to validate banking ids, such as ISIN, LEI and SEDOL.

In [1]:
# Sets up the location of the api relative to this notebook 
import sys
sys.path.append('../../')

In [2]:
# Import the module for normalizing banking id information
from financial_entity_cleaner.id_cleaner import banking_id

In [3]:
# Create a CountryCleaner object
id_cleaner_obj=banking_id.BankingIdCleaner()

In [4]:
# Check the types of validation available
id_cleaner_obj.get_types_id_validations()

['lei', 'isin', 'sedol']

## 1. Basic usage

By default, the API assumes that the value passed as parameter is an ISIN code.
The API returns:
- None if the value is not a string or has no characters in it.
- True if the value is a valid ID of the specified type
- False if the value is not a valid ID of the specified type

In [ ]:
id_cleaner_obj.id_type

In [ ]:
# Testing a valid ISIN code
id_cleaner_obj.is_valid_id('GB00B1YW4409')

In [ ]:
# Testing an invalid ISIN code
id_cleaner_obj.is_valid_id('tttt0B1YW4409')

## 2. Working with other ID types (LEI and SEDOL)

In [ ]:
id_cleaner_obj.id_type='lei'

In [ ]:
# Testing a valid LEI code
id_cleaner_obj.is_valid_id('969500DPKGC9JE9F0820')

In [ ]:
id_cleaner_obj.id_type='sedol'

In [ ]:
# Testing a valid SEDOL code
id_cleaner_obj.is_valid_id('2595708')

The library throws an exception id the type is not supported.

In [ ]:
id_cleaner_obj.id_type='test'

## 3. Cleaning and Validating

The library can also be used for cleaning and validation at the same time. In this case, it will return a list of values where list[0] indicates if the id is valid or not and list[1] returns the cleaner ID.

In [ ]:
id_cleaner_obj.id_type='lei'

In [ ]:
# Cleaning a valid LEI code
clean_lei = id_cleaner_obj.get_clean_id('969500DPKGC9JE9F0820')
clean_lei

By default, if the id is invalid get_clean_id() returns np.nan

In [ ]:
# Cleaning an invalid LEI code
clean_lei = id_cleaner_obj.get_clean_id('96XX00DPKGC9JE9F0820')
clean_lei

If this behaviour is not required, set the parameter set_null_for_invalid=False

In [ ]:
# Cleaning an invalid LEI code
clean_lei = id_cleaner_obj.get_clean_id('96XX00DPKGC9JE9F0820', False)
clean_lei